In [ ]:
pip install keras tensorflow pandas

In [ ]:
!pip install textblob

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
BASE_PATH = '/kaggle/input/llm-classification-finetuning'
test_data = pd.read_csv(f'{BASE_PATH}/test.csv')
train_data = pd.read_csv(f'{BASE_PATH}/train.csv')

train_data.head()


In [ ]:
print(train_data['prompt'].dtype)
print(train_data['response_a'].dtype)
print(train_data['response_b'].dtype)

train_data['prompt'] = train_data['prompt'].astype('string')
train_data['response_a'] = train_data['response_a'].astype('string')
train_data['response_b'] = train_data['response_b'].astype('string')


In [ ]:
import keras
import keras_hub
import numpy as np
import tensorflow as tf
import re
from textblob import TextBlob
from textblob.np_extractors import ConllExtractor
from textblob.sentiments import NaiveBayesAnalyzer
from textblob import TextBlob

import re

# Custom disagreement word list (you can expand this)
DISAGREEMENT_WORDS = [
    "no", "never", "wrong", "incorrect", "false", "not at all", "absolutely not",
    "unacceptable", "nonsense", "ridiculous", "invalid", "absurd", "flawed", "misleading",
    "inaccurate", "but", "however", "although", "though", "yet", "still", "instead",
    "whereas", "nonetheless", "dislike", "hate", "oppose", "reject", "dispute", "disagree",
    "criticize", "complain", "challenge", "resist", "object", "deny", "doubtful",
    "questionable", "unclear", "unsure", "maybe not", "possibly wrong", "doesn’t seem right",
    "suspicious", "sure...", "wow", "seriously?", "really?", "whatever", "cool story",
    "if you say so", "I guess", "I don’t think so", "I don’t agree", "That’s not true",
    "That doesn’t make sense", "I beg to differ", "That’s debatable", "That's incorrect"
]

def disagrability_score(text):
    blob = TextBlob(text.lower())

    # Count disagreement words/phrases
    dis_words_found = [word for word in DISAGREEMENT_WORDS if word in text.lower()]
    disagreement_count = len(dis_words_found)
    return disagreement_count/(len(blob))


def is_code(text: str) -> bool:
    if not isinstance(text, str) or len(text.strip()) == 0:
        return False  # Ignore non-strings or empty text

    code_patterns = [
        r"\bdef\b", r"\bclass\b", r"\bimport\b",              # Python
        r"\bpublic\b|\bprivate\b|\bstatic\b|\bnew\b",         # Java/C++
        r"#include\s*<.*>", r"::", r";\s*$",                  # C/C++
        r"console\.log", r"System\.out\.println",             # JS/Java
        r"<\/?\w+>",                                          # HTML/XML tags
        r"\bfunction\b", r"\bvar\b", r"\bconst\b", r"=>",     # JS
        r"\s{4,}\w+",                                         # Indented lines
        r"\{[\s\S]*?\}",                                      # Curly-brace blocks
        r"=\s*[^=]",                                          # Assignment ops
        r"\n.*\n.*\n",                                        # Multiline blocks
        r"\$\(", r"\bprint\s*\(",                             # Bash, Python
        r"^\s*#.*$",                                          # Comments
        r"\breturn\b", r"\btry\b", r"\bcatch\b",              # Control flow
        r"translate",
    ]

    match_count = sum(bool(re.search(pattern, text, flags=re.MULTILINE)) for pattern in code_patterns)
    
    return match_count >= 1  # Adjustable threshold


extractor = ConllExtractor()

def count_syllables(word):
    word = word.lower()
    vowels = "aeiouy"
    syllables = 0
    prev_char_was_vowel = False
    for char in word:
        if char in vowels:
            if not prev_char_was_vowel:
                syllables += 1
            prev_char_was_vowel = True
        else:
            prev_char_was_vowel = False
    if word.endswith("e"):
        syllables = max(1, syllables - 1)
    return max(1, syllables)

def readability_scores(text):
    # split sentences and words
    sentences = [s for s in re.split(r'[.!?]+', text) if s.strip()]
    words = re.findall(r'\w+', text)
    
    # ensure no zero counts
    num_sentences = max(1, len(sentences))
    num_words     = max(1, len(words))
    
    # count syllables
    syllable_count = sum(count_syllables(w) for w in words)
    
    # Flesch Reading Ease (0–100+)
    fre = (
        206.835
        - 1.015 * (num_words / num_sentences)
        - 84.6  * (syllable_count / num_words)
    )
    # return normalized 0–1 score
    return fre / 100.0

def find_relevance(A,B):
    blob = TextBlob(A,np_extractor=extractor)
    prompt=blob.noun_phrases
    
    blob = TextBlob(B, np_extractor=extractor)
    response=blob.noun_phrases
    
    set2= set(response)

    if not prompt:
        return 0.0
    
    not_in_list=[word for word in prompt if word not in set2]
    
    percent = (len(not_in_list) / len(prompt))
    return (1-percent)

def sentiment(A):
    text=str(A)
    blob=TextBlob(A)
    return blob.sentiment.polarity



In [ ]:
inputs = []
tokenized_winner_model = []

## Reducing the size the dataset to optimize the encoding process
for i in range(len(train_data)):
    inputa = np.zeros(5)

    inputa[0] = (find_relevance(train_data['prompt'].iloc[i],train_data['response_a'].iloc[i])-find_relevance(train_data['prompt'].iloc[i],train_data['response_b'].iloc[i]))
    inputa[1] = (readability_scores(train_data['response_a'].iloc[i])-readability_scores(train_data['response_b'].iloc[i]))
    inputa[2] = (len(train_data['response_a'].iloc[i])/len((train_data['response_b'].iloc[i])))
    inputa[3] = (sentiment((train_data['response_a'].iloc[i]))-sentiment((train_data['response_b'].iloc[i])))
    inputa[4] = (disagrability_score(train_data['response_a'].iloc[i]))-(disagrability_score(train_data['response_b'].iloc[i]))

    if not is_code(train_data['response_a'].iloc[i]) and not is_code(train_data['response_b'].iloc[i]):
        inputs.append(inputa)
        
        if train_data['winner_model_a'].iloc[i]==1:
            tokenized_winner_model.append(0)
        elif train_data['winner_model_b'].iloc[i]==1:
            tokenized_winner_model.append(1)
        else:
            tokenized_winner_model.append(2)


In [ ]:
print (len(tokenized_winner_model))

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Concatenate, Flatten
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.callbacks import LearningRateScheduler
import numpy as np
import tensorflow as tf

initial_learning_rate = 0.00001
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate,
    decay_steps=1000,        # after how many steps to decay
    decay_rate=0.96,         # multiply learning rate by this amount
    staircase=True           # whether to apply decay in discrete steps
)

optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule)


# === INPUTS ===
model = keras.Sequential ([
    keras.layers.Dense(8, input_shape=(5,), activation='relu'),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(16, activation='relu'),
    keras.layers.Dense(8, activation='relu'),
    keras.layers.Dense(4, activation='relu'),
    keras.layers.Dense(3, activation='sigmoid')
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


# === FIT THE MODEL ===
inputs_np = np.array(inputs)
tokenized_winner_model_np = np.array(tokenized_winner_model)



In [ ]:
# === FIT THE MODEL ===
model.fit(
    inputs_np,
    tokenized_winner_model_np,
    epochs=1000, 
    batch_size=32
)

In [ ]:
# Now use the vectorizer to transform your data
test_inputs = []

for i in range(len(test_data)):
    inputa = np.zeros(5)

    inputa[0] = (find_relevance(test_data['prompt'].iloc[i],test_data['response_a'].iloc[i])-find_relevance(test_data['prompt'].iloc[i],test_data['response_b'].iloc[i]))
    inputa[1] = (readability_scores(test_data['response_a'].iloc[i])-readability_scores(test_data['response_b'].iloc[i]))
    inputa[2] = (len(test_data['response_a'].iloc[i])/len((test_data['response_b'].iloc[i])))
    inputa[3] = (sentiment((test_data['response_a'].iloc[i]))-sentiment((test_data['response_b'].iloc[i])))
    inputa[4] = (disagrability_score(test_data['response_a'].iloc[i]))-(disagrability_score(test_data['response_b'].iloc[i]))

    test_inputs.append(inputa)    
    
test_inputs_np = np.array(test_inputs)

# Create a DataFrame for the submission
submission_df = pd.read_csv(f'{BASE_PATH}/sample_submission.csv')
submission_df['id'] = test_data['id']
submission_df['winner_model_a'] = 0.0
submission_df['winner_model_b'] = 0.0
submission_df['winner_tie'] = 0.0

# Fill the DataFrame with predictions
for i in range(len(test_data)):
    prediction = model.predict([
        test_inputs_np[i:i+1]
    ])
    
    submission_df.loc[i, 'winner_model_a'] = prediction[0][0]
    print(prediction[0][0])
    submission_df.loc[i, 'winner_model_b'] = prediction[0][1]
    print(prediction[0][1])
    submission_df.loc[i, 'winner_tie'] = prediction[0][2]
    print(prediction[0][2])

# Save to CSV file
submission_df.to_csv('submission.csv', index=False)